In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame(columns=['YR','TM','WON','TITLE_GAME','FINAL_FOUR','PTS','OPP_PTS','FG','FGA','3P','3PA','FT','FTA','ORB','TRB','AST','STL','BLK','TOV','PF','OPP_FG','OPP_FGA','OPP_3P','OPP_3PA','OPP_FT','OPP_FTA','OPP_ORB','OPP_TRB','OPP_AST','OPP_STL','OPP_BLK','OPP_TOV','OPP_PF'])
teams = pd.read_csv('./teams_2011on.csv') # sports-reference.com only has opposing team stats for individual games from 2011 onwards

In [3]:
driver = webdriver.Chrome('/Users/bpfalz/Downloads/chromedriver')

# Set the timeout to 10 seconds
timeout = 10

for i in range(len(teams)):
    for team in teams.iloc[i,1:]:
        temp_df = pd.DataFrame(columns=['DATE','COURT','OPP','WL','PTS','OPP_PTS','FG','FGA','FG%','3P','3PA','3P%','FT','FTA','FT%','ORB','TRB','AST','STL','BLK','TOV','PF','BLANK','OPP_FG','OPP_FGA','OPP_FG%','OPP_3P','OPP_3PA','OPP_3P%','OPP_FT','OPP_FTA','OPP_FT%','OPP_ORB','OPP_TRB','OPP_AST','OPP_STL','OPP_BLK','OPP_TOV','OPP_PF'])
        
        school = str(team)
        yr = str(teams.iloc[i,0])
        won = 1 if team == teams.iloc[i,1] else 0
        title_game = 1 if team == teams.iloc[i,1] or team == teams.iloc[i,2] else 0
        final_four = 1
        url = 'https://www.sports-reference.com/cbb/schools/' + school + '/men/' + yr + '-gamelogs.html'
        
        driver.get(url)

        try:
            # Wait for the elements to be present before trying to find them
            tm_stats = WebDriverWait(driver, timeout).until(EC.presence_of_all_elements_located((By.XPATH, '//tr/td')))
        except TimeoutException:
            print("Timed out waiting for element")

        stats = [yr,school,won,title_game,final_four]
        
        for j in range(1,7):
            if title_game == 0 and j == 6:
                continue
            data = []
            for k in range(len(tm_stats)-j*39,len(tm_stats)-(j-1)*39):
                data.append(tm_stats[k].text)
            temp_df.loc[len(temp_df)] = data

        temp_df = temp_df.apply(lambda x: pd.to_numeric(x) if x.name not in ['DATE','COURT','OPP','WL','FG%','3P%','FT%','OPP_FG%','OPP_3P%','OPP_FT%','BLANK'] else x)

        # Calculate the averages for numerical columns
        numerical_cols = temp_df.select_dtypes(include=np.number).columns
        averages = temp_df[numerical_cols].mean().round(1).values
        
        stats = np.append(stats,averages)

        print(stats)
        # Add the new row to the dataframe
        df.loc[len(df)] = stats

driver.close()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


['2023' 'connecticut' '1' '1' '1' '79.2' '59.2' '28.5' '58.7' '9.3' '23.5'
 '12.8' '16.0' '10.2' '39.5' '19.0' '4.8' '4.8' '11.7' '15.7' '20.0'
 '58.3' '5.2' '18.2' '14.0' '19.3' '8.3' '28.3' '9.2' '6.3' '1.7' '9.0'
 '14.8']
['2023' 'san-diego-state' '0' '1' '1' '66.2' '62.7' '24.7' '60.8' '5.7'
 '17.8' '11.2' '17.0' '10.5' '36.5' '10.2' '4.8' '4.7' '10.3' '17.0'
 '20.8' '56.2' '5.2' '22.2' '15.8' '19.3' '9.2' '33.5' '9.0' '5.7' '3.3'
 '10.7' '16.7']
['2023' 'miami-fl' '0' '0' '1' '76.8' '70.6' '26.2' '59.4' '7.0' '18.8'
 '17.4' '21.0' '9.4' '33.6' '11.4' '7.0' '2.0' '9.2' '13.2' '26.0' '59.8'
 '8.6' '26.2' '10.0' '14.0' '8.8' '31.6' '16.0' '5.4' '4.0' '11.4' '16.6']
['2023' 'florida-atlantic' '0' '0' '1' '71.2' '67.6' '24.8' '58.4' '8.6'
 '26.2' '13.0' '17.6' '11.8' '37.8' '13.0' '4.2' '3.2' '11.6' '14.8'
 '25.0' '61.4' '8.0' '23.2' '9.6' '14.8' '10.6' '31.6' '11.4' '6.4' '2.0'
 '9.0' '16.6']
['2022' 'kansas' '1' '1' '1' '76.2' '62.2' '28.3' '61.3' '7.2' '18.2'
 '12.3' '18.3' '9.0' '3

In [9]:
df = df.apply(lambda x: pd.to_numeric(x) if x.name != 'TM' else x)

In [11]:
df.dtypes

YR              int64
TM             object
WON             int64
TITLE_GAME      int64
FINAL_FOUR      int64
PTS           float64
OPP_PTS       float64
FG            float64
FGA           float64
3P            float64
3PA           float64
FT            float64
FTA           float64
ORB           float64
TRB           float64
AST           float64
STL           float64
BLK           float64
TOV           float64
PF            float64
OPP_FG        float64
OPP_FGA       float64
OPP_3P        float64
OPP_3PA       float64
OPP_FT        float64
OPP_FTA       float64
OPP_ORB       float64
OPP_TRB       float64
OPP_AST       float64
OPP_STL       float64
OPP_BLK       float64
OPP_TOV       float64
OPP_PF        float64
dtype: object

In [25]:
# Calculated fields -- interaction between team stats and opposition stats

df['FG%'] = df['FG'] / df['FGA']
df['2P'] = df['FG'] - df['3P']
df['2PA'] = df['FGA'] - df['3PA']
df['2P%'] = df['2P'] / df['2PA']
df['3P%'] = df['3P'] / df['3PA']
df['eFG%'] = (df['FG'] + 0.5*df['3P']) / df['FGA']
df['DRB'] = df['TRB'] - df['ORB']
df['OPP_2P'] = df['OPP_FG'] - df['OPP_3P']
df['OPP_2PA'] = df['OPP_FGA'] - df['OPP_3PA']
df['OPP_FG%'] = df['OPP_FG'] / df['OPP_FGA']
df['OPP_2P%'] = df['OPP_2P'] / df['OPP_2PA']
df['OPP_3P%'] = df['OPP_3P'] / df['OPP_3PA']
df['OPP_eFG%'] = (df['OPP_FG'] + 0.5*df['OPP_3P']) / df['OPP_FGA']
df['OPP_DRB'] = df['OPP_TRB'] - df['OPP_ORB']
df['PTS_DIFF'] = df['PTS'] - df['OPP_PTS']
df['PTS/OPP_PTS'] = df['PTS'] / df['OPP_PTS']
df['AST/TOV'] = df['AST'] / df['TOV']
df['AST/OPP_AST'] = df['AST'] / df['OPP_AST']
df['TRB_DIFF'] = df['TRB'] - df['OPP_TRB']
df['TRB%'] = df['TRB'] / (df['OPP_TRB'] + df['TRB'])
df['DRB%'] = df['DRB'] / (df['OPP_ORB'] + df['DRB'])
df['ORB%'] = df['ORB'] / (df['OPP_DRB'] + df['ORB'])
df['FG%/OPP_FG%'] = df['FG%'] / df['OPP_FG%']
df['2P%/OPP_2P%'] = df['2P%'] / df['OPP_2P%']
df['3P%/OPP_3P%'] = df['2P%'] / df['OPP_3P%']
df['eFG%/OPP_eFG%'] = df['eFG%'] / df['OPP_eFG%']
df['3PA/FGA'] = df['3PA'] / df['FGA']
df['2P/OPP_2P'] = df['2P'] / df['OPP_2P']
df['3P/OPP_3P'] = df['3P'] / df['OPP_3P']
df['FG/OPP_FG'] = df['FG'] / df['OPP_FG']
df['FT/OPP_FT'] = df['FT'] / df['OPP_FT']
df['2PA/OPP_2PA'] = df['2PA'] / df['OPP_2PA']
df['3PA/OPP_3PA'] = df['3PA'] / df['OPP_3PA']
df['FGA/OPP_FGA'] = df['FGA'] / df['OPP_FGA']
df['FTA/OPP_FTA'] = df['FTA'] / df['OPP_FTA']
df['STL/OPP_STL'] = df['STL'] / df['OPP_STL']
df['BLK/OPP_BLK'] = df['BLK'] / df['OPP_BLK']
df['TOV/OPP_TOV'] = df['TOV'] / df['OPP_TOV']
df['PF/OPP_PF'] = df['PF'] / df['OPP_PF']
df['FGA_DIFF'] = df['FGA'] - df['OPP_FGA']

In [ ]:
pd.set_option('display.max_columns', None)

In [26]:
df

,YR,TM,WON,TITLE_GAME,FINAL_FOUR,PTS,OPP_PTS,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF,OPP_FG,OPP_FGA,OPP_3P,OPP_3PA,OPP_FT,OPP_FTA,OPP_ORB,OPP_TRB,OPP_AST,OPP_STL,OPP_BLK,OPP_TOV,OPP_PF,FG%,2P,2PA,2P%,3P%,DRB,OPP_2P,OPP_2PA,OPP_FG%,OPP_2P%,OPP_3P%,OPP_DRB,PTS_DIFF,PTS/OPP_PTS,AST/TOV,AST/OPP_AST,TRB_DIFF,TRB%,DRB%,ORB%,BLK/OPP_BLK,FG%/OPP_FG%,2P%/OPP_2P%,3P%/OPP_3P%,3PA/FGA,FGA/OPP_FGA,FGA_DIFF,eFG%,OPP_eFG%,eFG%/OPP_eFG%,2P/OPP_2P,3P/OPP_3P,FG/OPP_FG,FT/OPP_FT,2PA/OPP_2PA,3PA/OPP_3PA,FTA/OPP_FTA,STL/OPP_STL,TOV/OPP_TOV,PF/OPP_PF
0,2023,connecticut,1,1,1,79.2,59.2,28.5,58.7,9.3,23.5,12.8,16.0,10.2,39.5,19.0,4.8,4.8,11.7,15.7,20.0,58.3,5.2,18.2,14.0,19.3,8.3,28.3,9.2,6.3,1.7,9.0,14.8,0.485520,19.2,35.2,0.545455,0.395745,29.3,14.8,40.1,0.343053,0.369077,0.285714,20.0,20.0,1.337838,1.623932,2.065217,11.2,0.582596,0.779255,0.337748,2.823529,1.415290,1.477887,1.909091,0.400341,1.006861,0.4,0.564736,0.387650,1.456819,1.297297,1.788462,1.425000,0.914286,0.877805,1.291209,0.829016,0.761905,1.300000,1.060811
1,2023,san-diego-state,0,1,1,66.2,62.7,24.7,60.8,5.7,17.8,11.2,17.0,10.5,36.5,10.2,4.8,4.7,10.3,17.0,20.8,56.2,5.2,22.2,15.8,19.3,9.2,33.5,9.0,5.7,3.3,10.7,16.7,0.406250,19.0,43.0,0.441860,0.320225,26.0,15.6,34.0,0.370107,0.458824,0.234234,24.3,3.5,1.055821,0.990291,1.133333,3.0,0.521429,0.738636,0.301724,1.424242,1.097656,0.963029,1.886404,0.292763,1.081851,4.6,0.453125,0.416370,1.088275,1.217949,1.096154,1.187500,0.708861,1.264706,0.801802,0.880829,0.842105,0.962617,1.017964
2,2023,miami-fl,0,0,1,76.8,70.6,26.2,59.4,7.0,18.8,17.4,21.0,9.4,33.6,11.4,7.0,2.0,9.2,13.2,26.0,59.8,8.6,26.2,10.0,14.0,8.8,31.6,16.0,5.4,4.0,11.4,16.6,0.441077,19.2,40.6,0.472906,0.372340,24.2,17.4,33.6,0.434783,0.517857,0.328244,22.8,6.2,1.087819,1.239130,0.712500,2.0,0.515337,0.733333,0.291925,0.500000,1.014478,0.913199,1.440715,0.316498,0.993311,-0.4,0.500000,0.506689,0.986799,1.103448,0.813953,1.007692,1.740000,1.208333,0.717557,1.500000,1.296296,0.807018,0.795181
3,2023,florida-atlantic,0,0,1,71.2,67.6,24.8,58.4,8.6,26.2,13.0,17.6,11.8,37.8,13.0,4.2,3.2,11.6,14.8,25.0,61.4,8.0,23.2,9.6,14.8,10.6,31.6,11.4,6.4,2.0,9.0,16.6,0.424658,16.2,32.2,0.503106,0.328244,26.0,17.0,38.2,0.407166,0.445026,0.344828,21.0,3.6,1.053254,1.120690,1.140351,6.2,0.544669,0.710383,0.359756,1.600000,1.042959,1.130508,1.459006,0.448630,0.951140,-3.0,0.498288,0.472313,1.054995,0.952941,1.075000,0.992000,1.354167,0.842932,1.129310,1.189189,0.656250,1.288889,0.891566
4,2022,kansas,1,1,1,76.2,62.2,28.3,61.3,7.2,18.2,12.3,18.3,9.0,38.2,14.7,5.5,5.2,9.7,16.2,21.2,61.7,6.7,23.3,13.2,16.8,9.8,32.2,11.3,4.7,3.8,10.8,15.8,0.461664,21.1,43.1,0.489559,0.395604,29.2,14.5,38.4,0.343598,0.377604,0.287554,22.4,14.0,1.225080,1.515464,1.300885,6.0,0.542614,0.748718,0.286624,1.368421,1.343616,1.296488,1.702497,0.296900,0.993517,-0.4,0.520392,0.397893,1.307868,1.455172,1.074627,1.334906,0.931818,1.122396,0.781116,1.089286,1.170213,0.898148,1.025316
5,2022,north-carolina,0,1,1,80.0,68.8,27.3,65.8,9.2,27.0,16.2,22.2,13.2,44.7,15.5,3.8,4.3,10.8,15.0,26.5,69.0,6.2,22.8,9.7,14.5,9.0,32.8,11.8,6.5,5.3,7.7,17.5,0.414894,18.1,38.8,0.466495,0.340741,31.5,20.3,46.2,0.384058,0.439394,0.271930,23.8,11.2,1.162791,1.435185,1.313559,11.9,0.576774,0.777778,0.356757,0.811321,1.080289,1.061678,1.715497,0.410334,0.953623,-3.2,0.484802,0.428986,1.130114,0.891626,1.483871,1.030189,1.670103,0.839827,1.184211,1.531034,0.584615,1.402597,0.857143
6,2022,duke,0,0,1,79.2,72.0,29.8,58.6,6.0,17.2,13.6,17.0,7.8,31.4,13.4,5.8,6.2,10.0,13.6,27.0,64.2,7.6,20.8,10.4,12.8,11.2,32.2,11.2,4.6,3.8,8.8,15.4,0.508532,23.8,41.4,0.574879,0.348837,23.6,19.4,43.4,0.420561,0.447005,0.365385,21.0,7.2,1.100000,1.340000,1.196429,-0.8,0.493711,0.678161,0.270833,1.631579,1.209177,1.286070,1.573354,0.293515,0.912773,-5.6,0.559727,0.479751,1.166704,1.226804,0.789474,1.103704,1.307692,0.953917,0.826923,1.328125,1.260870,1.136364,0.883117
7,2022,villanova,0,0,1,65.8,60.2,22.0,55.0,9.6,26.6,12.2,13.8,9.6,31.4,10.6,4.6,1.8,8.6,12.6,22.6,55.8,6.0,

In [36]:
df.set_index(['YR', 'TM'], inplace=True)
df

WON  TITLE_GAME  FINAL_FOUR   PTS  OPP_PTS    FG  \
YR   TM                                                                        
2023 connecticut              1           1           1  79.2     59.2  28.5   
     san-diego-state          0           1           1  66.2     62.7  24.7   
     miami-fl                 0           0           1  76.8     70.6  26.2   
     florida-atlantic         0           0           1  71.2     67.6  24.8   
2022 kansas                   1           1           1  76.2     62.2  28.3   
     north-carolina           0           1           1  80.0     68.8  27.3   
     duke                     0           0           1  79.2     72.0  29.8   
     villanova                0           0           1  65.8     60.2  22.0   
2021 baylor                   1           1           1  77.0     61.7  28.5   
     gonzaga                  0           1           1  86.0     72.2  31.8   
     houston                  0           0           1  67.6     60.2  23.0   
     ucla                     0           0           1  73.8     65.8  27.8   
2019 virginia                 1           1           1  69.2     61.7  25.7   
     texas-tech               0           1           1  71.0     60.7  25.5   
     auburn                   0           0           1  80.6     73.2  28.2   
     michigan-state           0           0           1  69.0     61.2  25.4   
2018 villanova                1           1           1  83.8     66.2  27.5   
     michigan                 0           1           1  68.8     62.0  24.8   
     kansas                   0           0           1  80.6     78.2  28.2   
     loyola-il                0           0           1  66.2     64.6  25.2   
2017 north-carolina           1           1           1  81.7     70.5  28.5   
     gonzaga                  0           1           1  71.8     63.3  24.5   
     oregon                   0           0           1  77.4     70.8  27.4   
     south-carolina           0           0           1  80.2     70.2  27.4   
2016 villanova                1           1           1  83.5     62.8  30.2   
     north-carolina           0           1           1  85.7     72.7  31.3   
     oklahoma                 0           0           1  75.0     75.0  26.8   
     syracuse                 0           0           1  68.4     61.2  23.6   
2015 duke                     1           1           1  71.8     56.3  25.8   
     wisconsin                0           1           1  76.0     69.8  25.7   
     kentucky                 0           0           1  70.6     56.6  24.6   
     michigan-state           0           0           1  65.8     65.2  22.6   
2014 connecticut              1           1           1  71.7     63.8  23.7   
     kentucky                 0           1           1  68.5     66.5  24.7   
     florida                  0           0           1  64.4     56.6  23.2   
     wisconsin                0           0           1  73.2     60.2  25.6   
2013 louisville               1           1           1  79.5     63.3  28.2   
     michigan                 0           1           1  75.3     65.2  28.3   
     wichita-state            0           0           1  71.8     64.2  23.6   
     syracuse                 0           0           1  63.8     48.8  21.6   
2012 kentucky                 1           1           1  81.3     69.5  27.8   
     kansas                   0           1           1  65.2     60.5  24.0   
     louisville               0           0           1  63.6     59.8  23.0   
     ohio-state               0           0           1  74.2     65.0  23.8   
2011 connecticut              1           1           1  66.3     56.0  23.8   
     butler                   0           1           1  62.8     61.3  20.3   
     kentucky                 0           0           1  64.6     61.0  23.6   
     virginia-commonwealth    0           0           1  74.6     66.8  24.6   

                           

In [49]:
df.to_csv('ff_team_pg_tourney.csv')

In [44]:
df.columns

Index(['WON', 'TITLE_GAME', 'FINAL_FOUR', 'PTS', 'OPP_PTS', 'FG', 'FGA', '3P',
       '3PA', 'FT', 'FTA', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'OPP_FG', 'OPP_FGA', 'OPP_3P', 'OPP_3PA', 'OPP_FT', 'OPP_FTA',
       'OPP_ORB', 'OPP_TRB', 'OPP_AST', 'OPP_STL', 'OPP_BLK', 'OPP_TOV',
       'OPP_PF', 'FG%', '2P', '2PA', '2P%', '3P%', 'DRB', 'OPP_2P', 'OPP_2PA',
       'OPP_FG%', 'OPP_2P%', 'OPP_3P%', 'OPP_DRB', 'PTS_DIFF', 'PTS/OPP_PTS',
       'AST/TOV', 'AST/OPP_AST', 'TRB_DIFF', 'TRB%', 'DRB%', 'ORB%',
       'BLK/OPP_BLK', 'FG%/OPP_FG%', '2P%/OPP_2P%', '3P%/OPP_3P%', '3PA/FGA',
       'FGA/OPP_FGA', 'FGA_DIFF', 'eFG%', 'OPP_eFG%', 'eFG%/OPP_eFG%',
       '2P/OPP_2P', '3P/OPP_3P', 'FG/OPP_FG', 'FT/OPP_FT', '2PA/OPP_2PA',
       '3PA/OPP_3PA', 'FTA/OPP_FTA', 'STL/OPP_STL', 'TOV/OPP_TOV',
       'PF/OPP_PF'],
      dtype='object')

In [46]:
df_ranks1 = df[['PTS', 'FG', 'FGA', '3P','3PA', 'FT', 'FTA', 'ORB', 'TRB', 'AST', 'STL', 'BLK','OPP_TOV','OPP_PF', 'FG%', '2P', '2PA', '2P%', '3P%', 'DRB','PTS/OPP_PTS','AST/TOV', 'AST/OPP_AST','TRB%', 'DRB%', 'ORB%','BLK/OPP_BLK', 'FG%/OPP_FG%', '2P%/OPP_2P%', '3P%/OPP_3P%','FGA/OPP_FGA','eFG%','eFG%/OPP_eFG%','2P/OPP_2P', '3P/OPP_3P', 'FG/OPP_FG', 'FT/OPP_FT', '2PA/OPP_2PA','3PA/OPP_3PA', 'FTA/OPP_FTA', 'STL/OPP_STL',]].copy().rank(method='min', ascending = False)

df_ranks2 = df[['TOV', 'PF','OPP_PTS', 'OPP_FG', 'OPP_FGA', 'OPP_3P', 'OPP_3PA', 'OPP_FT', 'OPP_FTA','OPP_ORB', 'OPP_TRB', 'OPP_AST', 'OPP_STL', 'OPP_BLK','OPP_2P', 'OPP_2PA','OPP_FG%', 'OPP_2P%', 'OPP_3P%', 'OPP_DRB','TOV/OPP_TOV','PF/OPP_PF']].copy().rank(method='min', ascending = True)

df_ranks = pd.merge(df_ranks1, df_ranks2, left_index=True, right_index=True)
df_ranks

PTS    FG   FGA    3P   3PA    FT   FTA   ORB  \
YR   TM                                                                      
2023 connecticut            12.0   5.0  16.0   6.0  12.0  33.0  41.0  17.0   
     san-diego-state        39.0  30.0   7.0  38.0  35.0  42.0  35.0  16.0   
     miami-fl               16.0  20.0  12.0  28.0  31.0   8.0  13.0  29.0   
     florida-atlantic       29.0  28.0  19.0  10.0   6.0  31.0  32.0   7.0   
2022 kansas                 17.0   8.0   6.0  26.0  33.0  36.0  28.0  32.0   
     north-carolina         10.0  18.0   1.0   8.0   3.0  11.0  10.0   4.0   
     duke                   12.0   4.0  18.0  37.0  36.0  28.0  35.0  39.0   
     villanova              41.0  46.0  32.0   4.0   5.0  38.0  45.0  22.0   
2021 baylor                 15.0   5.0   3.0  11.0  15.0  41.0  44.0  13.0   
     gonzaga                 1.0   1.0  15.0  14.0  14.0  22.0  26.0  41.0   
     houston                37.0  43.0  12.0  19.0   8.0  23.0  16.0   1.0   
     ucla                   23.0  13.0   9.0  26.0  27.0  43.0  40.0  33.0   
2019 virginia               32.0  22.0  25.0  14.0   7.0  48.0  47.0  21.0   
     texas-tech             30.0  25.0  27.0  22.0  18.0  35.0  39.0  46.0   
     auburn                  7.0  10.0   7.0   2.0   2.0  34.0  30.0  40.0   
     michigan-state         33.0  26.0  23.0  25.0  15.0  44.0  45.0  22.0   
2018 villanova               3.0  15.0  20.0   1.0   1.0  11.0  19.0  17.0   
     michigan               34.0  28.0  31.0  31.0  10.0  36.0  28.0  43.0   
     kansas                  7.0  10.0   5.0   8.0  13.0  16.0  18.0  29.0   
     loyola-il              39.0  27.0  48.0  35.0  38.0  47.0  47.0  48.0   
2017 north-carolina          5.0   5.0   2.0  35.0  20.0   4.0   2.0   2.0   
     gonzaga                25.0  34.0  21.0  22.0  26.0  15.0   6.0  20.0   
     oregon                 14.0  16.0  22.0  13.0  17.0  19.0  19.0  19.0   
     south-carolina          9.0  16.0  10.0  34.0  29.0   3.0   5.0  15.0   
2016 villanova               4.0   3.0  45.0   6.0  32.0  23.0  35.0  46.0   
     north-carolina          2.0   2.0   3.0  31.0  37.0  11.0  16.0   3.0   
     oklahoma               20.0  19.0  12.0   4.0   9.0  40.0  33.0  11.0   
     syracuse               36.0  39.0  23.0  28.0  23.0  19.0  19.0   6.0   
2015 duke                   25.0  21.0  45.0  39.0  43.0  18.0  23.0  41.0   
     wisconsin              18.0  22.0  39.0  11.0  19.0  11.0  12.0  37.0   
     kentucky               31.0  32.0  40.0  48.0  48.0   5.0   8.0  31.0   
     michigan-state         41.0  45.0  38.0  20.0  23.0  31.0  15.0  38.0   
2014 connecticut            28.0  38.0  44.0  22.0  28.0   9.0  25.0  45.0   
     kentucky               35.0  30.0  41.0  42.0  45.0  19.0  13.0  10.0   
     florida                45.0  42.0  43.0  45.0  40.0  28.0  27.0  22.0   
     wisconsin              24.0  24.0  33.0  17.0  21.0  23.0  30.0  35.0   
2013 louisville             11.0  10.0  36.0  41.0  41.0   5.0   3.0  13.0   
     michigan               19.0   8.0  16.0  17.0  22.0  45.0  42.0  26.0   
     wichita-state          25.0  39.0  27.0  28.0  23.0   7.0   9.0   9.0   
     syracuse               46.0  47.0  47.0  46.0  46.0  10.0   7.0  33.0   
2012 kentucky                6.0  13.0  37.0  42.0  47.0   1.0   1.0  26.0   
     kansas                 43.0  35.0  11.0  47.0  42.0  30.0  19.0   8.0   
     louisville             47.0  43.0  25.0  40.0  43.0  38.0  33.0  22.0   
     ohio-state             22.0  36.0  30.0  31.0  34.0   2.0   4.0  11.0   
2011 connecticut            38.0  36.0  35.0  42.0  39.0  27.0  38.0  26.0   
     butler                 48.0  48.0  29.0  14.0   4.0  23.0  23.0   5.0   
     kentucky               44.0  39.0  42.0  20.0  30.0  46.0  43.0  35.0   
     virginia-commonwealth  21.0  32.0  34.0   3.0  10.0  16.0  11.0  43.0   

                             TRB   AST   STL   BLK  OPP_TOV  OPP_PF   FG%  \
YR   TM                                            

In [48]:
average_ranks = df_ranks.mean(axis=1)
average_ranks.sort_values()

YR    TM                   
2023  connecticut              16.904762
2015  duke                     18.174603
2013  louisville               18.523810
2016  villanova                18.761905
2015  kentucky                 19.460317
2021  baylor                   19.492063
2016  north-carolina           19.619048
2018  villanova                20.269841
2017  north-carolina           20.301587
2019  virginia                 20.619048
2022  kansas                   20.761905
2013  syracuse                 20.984127
2021  houston                  21.269841
2014  wisconsin                21.396825
2012  kentucky                 21.460317
2017  south-carolina           21.650794
2021  gonzaga                  22.412698
2019  texas-tech               22.857143
2013  michigan                 23.047619
2019  michigan-state           23.476190
2014  florida                  23.492063
2011  connecticut              23.507937
2012  ohio-state               23.984127
2013  wichita-state          